In [1]:
import sys
import os
import torch
import yaml
import torch.nn as nn

root_path = ""
for path in os.getcwd().split("\\")[:-2]:
    root_path += f"{path}/"
sys.path.insert(1, root_path)

In [2]:
import config
import src.utils as utils
from src.model.rnn import RNNModel
import src.trainer as trainer

In [ ]:
num_epoch = 100

In [ ]:
# Init device
print("Initaling device ...")
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Load data
print("Loading data ...")
corpus_path = os.path.join(config.ROOT_PATH, "data/raw/pentext.pt")
corpus = torch.load(corpus_path)

# Init model
print("Initaling model ...")
with open(os.path.join(config.ROOT_PATH, "config/model_param.yaml"), "r") as f:
    model_param = yaml.safe_load(f)
f.close()
model_param["ntoken"] = corpus.vocab_size
model = RNNModel(**model_param)

# Train model
print("Starting train model ...")
with open(os.path.join(config.ROOT_PATH, "config/trainer_param.yaml"), "r") as f:
    trainer_params = yaml.safe_load(f)
f.close()


train_data = utils.batchify(
    data=corpus.train, batch_size=trainer_params["batch_size"]
)
# valid_data = utils.batchify(data=corpus.valid, batch_size=trainer_params["batch_size"])
# test_data = utils.batchify(data=corpus.test, batch_size=trainer_params["batch_size"])

trainer_params["model"] = model.to(device)
trainer_params["train_data"] = train_data.to(device)
trainer_params["n_token"] = corpus.vocab_size
trainer_params["criterion"] = nn.CrossEntropyLoss()

for epoch in range(1, num_epoch + 1):
    trainer.train_each_epoch(**trainer_params)